In [21]:
!pip install selenium
!pip install beautifulsoup4

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import re

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
driver.get("https://gisapp.adcogov.org/PropertySearch")

# Wait for the search box to be visible and locate it
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "search-text"))
)

# Send search query
search_box.send_keys("7471 E 157th Ave")
search_box.send_keys(Keys.RETURN)

# Wait for the table row containing the Parcel Number to be visible and clickable
parcel_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//table[@class='table']//tr[2]//td[1]//a"))
)

# Click the Parcel Number link which opens a new tab
parcel_link.click()

# Wait for the new tab to open (wait until the number of windows (tabs) becomes 2)
WebDriverWait(driver, 3).until(EC.number_of_windows_to_be(2))

# Get all window handles (tabs)
windows = driver.window_handles

# Switch to the new tab (the last window handle)
driver.switch_to.window(windows[-1])

# Extract the parcel number from the new page (you can adjust the XPath as needed)
parcel_number = driver.current_url.split('pid=')[1]

# Construct the new URL for scraping
new_url = f"https://gisapp.adcogov.org/QuickSearch/doreport.aspx?pid={parcel_number}"

# Now, go to the new URL and scrape the data
driver.get(new_url)

# Wait for the page to load fully
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "body"))
)

# Extract page source for scraping
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

### Parcel ID And Owner Information

In [23]:
# Extract the Parcel Number using regex (13 digits)
parcel_number_text = None
parcel_number = soup.find('span', {'class': 'ParcelIDAndOwnerInformation'})
if parcel_number:
    # Search for a 13-digit number in the text
    match = re.search(r'\d{13}', parcel_number.text.strip())
    if match:
        parcel_number_text = match.group(0)  # Extract the matched 13-digit number
    else:
        parcel_number_text = "Parcel Number Not Found"
else:
    parcel_number_text = "Parcel Number Not Found"

# Extract Owner's Name
owner_name = soup.find('span', {'id': 'ownerNameLabel'}).text.strip()

# Extract Property Address
property_address = soup.find('td', {'id': 'propertyContentCell'}).text.strip()

# Create a dictionary to store the data
property_data = {
    "Parcel Number": parcel_number_text,
    "Owner Name": owner_name,
    "Property Address": property_address
}

# Output the dictionary
for key, value in property_data.items():
    print(f"{key}: {value}")

Parcel Number: 0157109207009
Owner Name: MAQUEDA ARMANDO
Property Address: 7471 E 157TH AVE  THORNTON CO


### Account Summary

In [24]:
account_summary_data = {}

# Extract Legal Description
legal_description_section = soup.find('div', {'id': 'Panel'})
if legal_description_section:
    legal_description = legal_description_section.find_next('div', {'class': 'SingleValueBoxElement'})
    if legal_description:
        legal_description = legal_description.text.strip()

# Extract Subdivision Plat
subdivision_plat_section = soup.find('span', string="Subdivision Plat")
if subdivision_plat_section:
    subdivision_plat = subdivision_plat_section.find_next('div', {'class': 'SingleValueBoxElement'})
    if subdivision_plat:
        subdivision_plat = subdivision_plat.text.strip()

# Extract Account Summary Table
account_summary_table = soup.find('div', {'class': 'TaxAccountSummary'}).find('table')

if account_summary_table:
    # Extract row data from the table
    rows = account_summary_table.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        account_summary_data["Account Number"] = columns[0].text.strip()
        account_summary_data["Date Added"] = columns[1].text.strip()
        account_summary_data["Tax District"] = columns[2].text.strip()
        account_summary_data["Mill Levy"] = columns[3].text.strip()

# Create a dictionary to store all the data
property_data = {
    "Legal Description": legal_description if legal_description else "",
    "Subdivision Plat": subdivision_plat if subdivision_plat else "",
    "Account Summary": account_summary_data if account_summary_data else ""
}

# Output the dictionary, printing each item on a new line
for key, value in property_data.items():
    if isinstance(value, dict): 
        print(f"{key}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"{key}: {value}")

Legal Description: TALON VIEW SUBDIVISION BLK 11 LOT 16
Subdivision Plat: TALON VIEW
Account Summary:
  Account Number: R0179391
  Date Added: 07/19/2010
  Tax District: 528
  Mill Levy: 199.526


### Permits

In [25]:
permits = {}

permit_cases_section = soup.find('span', string="Permit Cases")
permit_cases_value = "Not Available"  # Default value

if permit_cases_section:
    permit_cases_value = permit_cases_section.find_next('div', {'class': 'MultiValueBoxElement'})
    if permit_cases_value:
        permit_cases_value = permit_cases_value.text.strip()


permits["Permit Cases"] = permit_cases_value

# Output the dictionary
for key, value in permits.items():
    if isinstance(value, dict): 
        print(f"{key}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"{key}: {value}")

Permit Cases: N/A


### Sales Summary

In [26]:
sales_table = soup.find('table', rules='all', border="2", style="border-width:2px;border-style:Double;width:100%;page-break-inside:avoid;")  # Or use 'class_' or 'id' based on the HTML structure

rows = sales_table.find_all('tr')[1:]  # Skipping the header row


sale_data = []
for row in rows:
    # Find all table data (td) cells within the row
    cells = row.find_all('td')
    
    # Check if the row has the correct number of columns (10 in this case)
    if len(cells) == 10:
        sale_dict = {
            'Sale Date': cells[0].find('span').get_text(strip=True) if cells[0].find('span') else '',
            'Sale Price': cells[1].find('span').get_text(strip=True) if cells[1].find('span') else '',
            'Deed Type': cells[2].find('span').get_text(strip=True) if cells[2].find('span') else '',
            'Reception Number': cells[3].find('span').get_text(strip=True) if cells[3].find('span') else '',
            'Book': cells[4].find('span').get_text(strip=True) if cells[4].find('span') else '',
            'Page': cells[5].find('span').get_text(strip=True) if cells[5].find('span') else '',
            'Grantor': cells[6].find('span').get_text(strip=True) if cells[6].find('span') else '',
            'Grantee': cells[7].find('span').get_text(strip=True) if cells[7].find('span') else '',
            'Doc. Fee': cells[8].find('span').get_text(strip=True) if cells[8].find('span') else '',
            'Doc. Date': cells[9].find('span').get_text(strip=True) if cells[9].find('span') else ''
        }
        
        sale_data.append(sale_dict)

if sale_data:
    for sale in sale_data:
        for key, value in sale.items():
            print(f"{key}: {value}")
        # add blank line between sales
        print()
else:
    print("No sale data found")

Sale Date: 10/23/2014
Sale Price: $0
Deed Type: BLK
Reception Number: 2014000078597
Book: 2014
Page: 
Grantor: COLORADO PROPERTY INVESTMENTS INC
Grantee: ELG INVESTORS LLC
Doc. Fee: $0
Doc. Date: 11/07/2014

Sale Date: 05/26/2017
Sale Price: $4,039,000.00
Deed Type: BLK
Reception Number: 2017000046039
Book: 
Page: 
Grantor: ELG INVESTORS LLC
Grantee: EQUINOX DEVELOPMENT LLC
Doc. Fee: $403.9
Doc. Date: 05/30/2017

Sale Date: 05/26/2017
Sale Price: $4,824,000.00
Deed Type: BLK
Reception Number: 2017000046079
Book: 
Page: 
Grantor: EQUINOX DEVELOPMENT LLC
Grantee: MELODY HOMES INC
Doc. Fee: $482.4
Doc. Date: 05/30/2017

Sale Date: 04/27/2020
Sale Price: $452,940.00
Deed Type: SWD
Reception Number: 2020000038465
Book: 
Page: 
Grantor: MELODY HOMES INC
Grantee: CALLISON MARC MAX AND, CALLISON RACHEL MICHELLE
Doc. Fee: $45.29
Doc. Date: 04/28/2020

Sale Date: 11/08/2024
Sale Price: $570,000.00
Deed Type: SWD
Reception Number: 2024000062692
Book: 
Page: 
Grantor: CALLISON MARC MAX AND, CALLIS

### Valuation Summary
* <span style="color: red;">missing Total Property Value & *Total Adjusted Value</span>


In [27]:
land_valuation_data = {}
land_valuation_section = soup.find('span', string="Land Valuation Summary")

if land_valuation_section:
    land_valuation_table = land_valuation_section.find_next('table', {'rules': 'all'})
    if land_valuation_table:
        rows = land_valuation_table.find_all('tr')[1:]  # Skip header row
        for row in rows:
            columns = row.find_all('td')
            
            if len(columns) > 0:
                if columns[0].text.strip() and columns[0].text.strip() != "Land Subtotal:":
                    land_valuation_data["Account Number"] = columns[0].text.strip()
                    land_valuation_data["Land Type"] = columns[1].text.strip()
                    land_valuation_data["Unit of Measure"] = columns[2].text.strip()
                    land_valuation_data["Number of Units"] = columns[3].text.strip()
                    land_valuation_data["Fire District"] = columns[4].text.strip()
                    land_valuation_data["School District"] = columns[5].text.strip()
                    land_valuation_data["Vacant/Improved"] = columns[6].text.strip()
                    land_valuation_data["Actual Value"] = columns[7].text.strip()
                    land_valuation_data["Assessed Value"] = columns[8].text.strip()

# Extract Improvements Valuation Summary
improvements_valuation_data = {}
improvements_valuation_section = soup.find('span', string="Improvements Valuation Summary")

if improvements_valuation_section:
    improvements_valuation_table = improvements_valuation_section.find_next('table', {'rules': 'all'})
    if improvements_valuation_table:
        rows = improvements_valuation_table.find_all('tr')[1:]  # Skip header row
        for row in rows:
            columns = row.find_all('td')
            
            if len(columns) > 1:
                # Only extract from valid rows with data (skip empty rows)
                if columns[0].text.strip() and columns[0].text.strip() != "Improvements Subtotal:":
                    improvements_valuation_data["Account Number"] = columns[0].text.strip()
                    improvements_valuation_data["Actual Value"] = columns[1].text.strip()
                    improvements_valuation_data["Assessed Value"] = columns[2].text.strip()

# # Extract Property Values (Total Property Value and Total Assessed Value)
# total_property_values = {}
# property_value_section = soup.find('span', string="Total Property Value")

# if property_value_section:
#     property_value_table = property_value_section.find_next('table')
#     if property_value_table:
#         rows = property_value_table.find_all('tr')
#         for row in rows:
#             columns = row.find_all('td')
            
#             if len(columns) > 1:
#                 if 'Total Property Value' in columns[0].text:
#                     total_property_values["Total Property Value"] = columns[1].text.strip()
#                     total_property_values["Total Assessed Value"] = columns[2].text.strip()

# # Extract Adjusted Actual and Assessed Values (if separate section exists)
# adjusted_values = {}
# adjusted_value_section = soup.find('span', string="Adjusted Actual Value")

# if adjusted_value_section:
#     adjusted_value_table = adjusted_value_section.find_next('table')
#     if adjusted_value_table:
#         rows = adjusted_value_table.find_all('tr')
#         for row in rows:
#             columns = row.find_all('td')
            
#             if len(columns) > 1:  # Ensure there are enough columns
#                 if "*Total Adjusted Value" in columns[0].text:
#                     adjusted_values["Adjusted Actual Value"] = columns[1].text.strip()
#                 if "*Total Adjusted Assessed Value" in columns[0].text:
#                     adjusted_values["Adjusted Assessed Value"] = columns[2].text.strip() if len(columns) > 2 else "Not Available"


valuation_data = {
    "Land Valuation Summary": land_valuation_data,
    "Improvements Valuation Summary": improvements_valuation_data,
    # "Total Property Value	": total_property_values,
    # "Adjusted Values": adjusted_values
}


for key, value in valuation_data.items():
    print()
    print(f"{key}:")
    if isinstance(value, dict):  # Check if the value is a dictionary
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"  {value}")


Land Valuation Summary:
  Account Number: R0179391
  Land Type: Residential
  Unit of Measure: Acres
  Number of Units: 0.2403
  Fire District: 
  School District: School District 27J-Brighton
  Vacant/Improved: I
  Actual Value: $105,000.00
  Assessed Value: $7,040.00

Improvements Valuation Summary:
  Account Number: R0179391
  Actual Value: $472,000.00
  Assessed Value: $31,620.00


### Building Summary

In [28]:
building_section = soup.find('span', {'class': 'BuildingSummary'})

building_data = {}

# Extract the table with the building details
table = building_section.find('table')

# Loop through each row in the table to extract key-value pairs
if table:
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 1:
            label = cells[0].get_text(strip=True)
            value = cells[1].get_text(strip=True)
            building_data[label] = value


for key, value in building_data.items():
    print(f"{key} {value}")

Built As: Ranch 1 Story
Year Built: 2019
Building Type: Residential
Construction Type: Frame Siding
Built As SQ Ft: 1889
Number of Rooms: 5
Number of Baths: 2.00
Number of Bedrooms: 3
Attached Garage SQ Ft: 651
Detached Garage Square Ft: 
Basement SQ Ft: 
Finished Basement SQ Ft: 


#### Tax Summary
* <span style="color: red;">need to combine with https://adcotax.com/</span>

### Enterprise Zone Summary

In [29]:
# Find the div containing the "Property within Enterprise Zone" section
enterprise_zone_section = soup.find('span', {'class': 'EnterpriseZoneSection'})

enterprise_zone_data = {}

# Extract the title of the section (e.g., "Property within Enterprise Zone")
title = enterprise_zone_section.find('span', {'id': 'Label'}).get_text(strip=True)

# Extract the value (True/False) from the "SingleValueBoxElement" div
value = enterprise_zone_section.find('div', {'class': 'SingleValueBoxElement'}).find('span').get_text(strip=True)

# Store the key-value pair in the dictionary
enterprise_zone_data[title] = value

# Output the results
for key, value in enterprise_zone_data.items():
    print(f"{key}: {value}")

Property within Enterprise Zone: False


### Precincts and Legislative Representatives Summary

In [30]:
representatives_data = {}

# Find all sections related to each representative type (Commissioner, State House, State Senate, US Congress)
representative_sections = soup.find_all('span', class_='PrecinctsLegislativeRepresentatives')

# Loop through each section and extract relevant data
for section in representative_sections:
    # Check if a title (span with id='Label') exists
    label_span = section.find('span', id='Label')
    if label_span:
        title = label_span.text.strip()  # Extract the title (e.g., "Commissioner Representative")
    else:
        title = "Unknown Representative"  # Default value if no title is found

    # Find the corresponding table within the section
    table = section.find('table')

    if table:  # Only proceed if the table exists
        # Loop through each row (skipping the header row)
        for row in table.find_all('tr')[1:]:  # Skipping the first header row
            columns = row.find_all('td')
            if len(columns) > 1:
                district = columns[0].text.strip()  # District number (or ID)
                link_to_rep = columns[1].find('a')['href']  # Representative's link

                # Store the data in the dictionary
                if title not in representatives_data:
                    representatives_data[title] = []
                representatives_data[title].append({
                    'District': district,
                    'Link': link_to_rep
                })

    # Check for the Precinct section and extract its value
    precinct_section = section.find('span', style="font-family:VERDANA, ARIAL, HELVETICA, SANS-SERIF;font-size:10pt;font-weight:normal;color:#000000;")
    if precinct_section:
        precinct_value = precinct_section.text.strip()
        if title not in representatives_data:
            representatives_data[title] = []
        representatives_data[title].append({
            'Precinct': precinct_value
        })

# Output the results
for title, reps in representatives_data.items():
    if title == "Unknown Representative":  # Skip the unknown representative section
        continue
    print(f"\n{title}:")
    for rep in reps:
        if 'District' in rep:  # Print district and link
            print(f"  District: {rep['District']}, Link: {rep['Link']}")
        if 'Precinct' in rep:  # Print precinct data
            print(f"  Precinct: {rep['Precinct']}")


Precinct:
  Precinct: 161

Commissioner Representative:
  District: 1, Link: https://gisportal.adcogov.org/rdr/comm_district1.htm

State House Representative:
  District: 33, Link: https://leg.colorado.gov/legislators/william-lindstedt

State Senate Representative:
  District: 24, Link: https://leg.colorado.gov/legislators/kyle-mullica

US Congress Representative:
  District: 8, Link: https://gabeevans.house.gov/


### Zoning Summary

In [31]:
zoning_section = soup.find('div', {'class': 'ZoningSummary'})

# Create an empty dictionary to store key-value pairs
zoning_data = {}

# Find the first table inside the zoning section
zoning_table = zoning_section.find('table')

# Iterate through the table rows to extract the zoning information
rows = zoning_table.find_all('tr')

# Skip the header row (first row), starting from the second row (index 1)
for row in rows[1:]:  # Skips the first row (index 0) which contains the headers
    # Find all columns (td tags) in the current row
    cols = row.find_all('td')
    
    # Ensure the row has exactly two columns (Zoning Authority and Zoning)
    if len(cols) == 2:
        # Extract the text from each column
        zoning_authority = cols[0].get_text(strip=True)  # Zoning Authority
        zoning = cols[1].get_text(strip=True)  # Zoning
        
        # Store the key-value pair in the dictionary
        zoning_data["Zoning Authority"] = zoning_authority
        zoning_data["Zoning"] = zoning

# Output the results as key-value pairs
for key, value in zoning_data.items():
    print(f"{key}: {value}")

Zoning Authority: THORNTON
Zoning: THORNTON
